# Cricket Simulator
1. Database search & get data
2. EDA
3. EDA Plots
4. Simulator for scores

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Global Variables & functions

In [5]:
p_names = {
    'ind': {
        'bt': ['ggambhir', 'vsehwag', 'rsharma', 'klrahul', 'vkohli', 'arahane', 'stendulkar', 'rdravid', 'sganguly'], 
        'wk': ['dhoni', 'dkartik', 'rpant'], 'all': ['hpandya', 'rjadeja', 'kpandya', 'ysingh'], 
        'bl': ['bkumar', 'rashwin', 'mdshami', 'bumrah', 'akumble', 'hsingh', 'isharma', 'zkhan', 'anehra']
    }, 
           
    'aus': {
        'bt': ['mhayden', 'rponting', 'mhussey', 'jlanger', 'ssmith'], 
        'wk': ['agilchrist'], 'all': ['mclarke', 'asymonds', 'abitchell'], 
        'bl': ['swarne', 'mstark', 'blee', 'gmcgrath']
    }, 
    
    'saf': {
        'bt': ['gkirsten', 'bdipenner', 'abdevilliers', 'hgibbs', 'jfaulkner', 'gsmith'], 
        'wk': ['mboucher'], 'all': ['spollock', 'kallis', 'lklusener'], 
        'bl': ['adonald', 'mntini', 'dsteyn', 'nboye']
    }, 
    
    'win': {
        'bt': ['cgayle', 'whinds', 'blara', 'schanderpaul', 'dsammy'], 
        'wk': ['npooran', 'dramdin'], 'all': ['pollard', 'dbravo', 'arussell', 'cbrathwaite'], 
        'bl': ['snarine', 'kroach', 'jholder', 'mdjason', 'ahosein']
    }, 
    
    'sri': {
        'bt': ['matapattu', 'mjayavardene', 'aranatunga', 'tdilshan', 'mahanama'], 
        'wk': ['sangakara', 'kaluwithrana'], 'all': ['sjayasuriya', 'adesilva'], 
        'bl': ['cvaas', 'mmurali', 'dfernando', 'yokumar', 'amendis']
    }
          }

In [6]:
def reset_wkts_proba(p_skill = 'bl'):
    temp_proba = {
        0: 0.96, 1: 0.04} if p_skill=='bl' else {
        0: 0.991, 1: 0.009} if p_skill=='all' else {
        0: 0.9991, 1: 0.0009} if p_skill == 'bt' else {
        0: 0.9999, 1: 0.0001}
    dict_sum = pd.Series(list(temp_proba.values())).sum()
    return {k: v/dict_sum for k, v in list(temp_proba.items())}  # normalize so sum of proba = 1

In [7]:
def reset_runs_proba(p_skill = 'bt'):
    temp_proba = {
    0: 0.55, 1: 0.25, 2: 0.1, 3: 0.005, 4: 0.1, 5: 0.001, 6: 0.05} if p_skill=='bt' else {
    0: 0.65, 1: 0.22, 2: 0.1, 3: 0.005, 4: 0.1, 5: 0.001, 6: 0.10} if p_skill=='all' else {
    0: 0.70, 1: 0.20, 2: 0.1, 3: 0.005, 4: 0.15, 5: 0.001, 6: 0.15} if p_skill == 'wk' else {
    0: 0.80, 1: 0.1, 2: 0.1, 3: 0.005, 4: 0.1, 5: 0.001, 6: 0.15}
    dict_sum = pd.Series(list(temp_proba.values())).sum()
    return {k: v/dict_sum for k, v in list(temp_proba.items())}  # normalize so sum of proba = 1

In [8]:
def reset_team_compos(team_name):
    compos_dict = {
        'ind': {'bt': 5, 'wk': 1, 'all': 2, 'bl': 3}, 
        'aus': {'bt': 4, 'wk': 1, 'all': 3, 'bl': 3}, 
        'saf': {'bt': 4, 'wk': 1, 'all': 2, 'bl': 4},
        'win': {'bt': 4, 'wk': 1, 'all': 2, 'bl': 4},
        'sri': {'bt': 3, 'wk': 2, 'all': 2, 'bl': 4}
    }

    return compos_dict[team_name]

# Class Definitions

### Player Class 

In [72]:
class Player(object):
    '''
    Class of Player object
    Attributes like player name, matches played, runs, wickets, wins, etc. 
    will be part of this class
    '''
    
    def __init__(self, p_name, team_name):
        self.p_name = p_name
        self.team_name = team_name
        self.p_skill = self.getSkill()  # batsman, bowler, wk
        self.m_played = 0
        self.bt_runs = 0
        self.bt_balls = 0
        self.bl_wkts = 0
        self.bl_runs = 0
        self.bl_balls = 0
        self.m_wins = 0
        self.runs_proba = reset_runs_proba(self.p_skill)
        self.wkts_proba = reset_wkts_proba(self.p_skill)
    
    
    def getSkill(self):
        key_list = list(p_names[self.team_name].keys())
        val_list = list(p_names[self.team_name].values())
        return [k for k, v_l in zip(key_list, val_list) for v in v_l if v==pl][0]
    
    
    def updateRunProba(self, run_in_ball):
        '''
        probability at which player scores runs
        default: reset_runs_proba(p_skill)
        '''
        for k, v in list(self.runs_proba.items()):
            if k == run_in_ball:
                if run_in_ball == 0:
                    self.runs_proba[k] = v*1.05
                else:
                    self.runs_proba[k] = v*1.2
            elif k > run_in_ball:
                self.runs_proba[k] = v*1.01
            else:
                self.runs_proba[k] = v
        
        dict_sum = pd.Series(list(self.runs_proba.values())).sum()
        return {k: v/dict_sum for k, v in list(self.runs_proba.items())}  # normalize so sum of proba = 1
    
    
    def simulateScore(self, wkts, balls, runs_proba, wkts_proba):
        '''
        simulate run score for this ball
        '''
        self.bt_balls += 1
        wkts_in_ball = np.random.choice(a = list(self.wkts_proba.keys()), p = pd.Series(list(self.wkts_proba.values())))

        if wkts_in_ball:
            run_in_ball = -1
        else: 
            run_in_ball = np.random.choice(a = list(self.runs_proba.keys()), p = pd.Series(list(self.runs_proba.values())))
            self.bt_runs += run_in_ball

            # runs proba updated as per run scored in prev ball
            self.runs_proba = self.updateRunProba(run_in_ball)

        return run_in_ball

### Team Class

In [73]:
class Team(Player):
    '''
    Class of Team object
    Attributes like team name, matches played, wins, etc. 
    will be part of this class
    '''
    
    def __init__(self, team_name):
        self.team_name = team_name
        self.m_played = 0
        self.m_wins = 0
        self.team_compos = reset_team_compos(team_name)
        self.players = self.generate_team()
        
        
    def generate_team(self):
        self.curr_team_d = {skill: list(np.random.choice(p_names[self.team_name][skill], 
                                                  self.team_compos[skill], 
                                                  replace = False))
                     for skill in ['bt', 'wk', 'all', 'bl']}

        curr_team = [p for sk in list(self.curr_team_d.keys()) for p in self.curr_team_d[sk]]
    
        return curr_team
    
    
    def update_team_comp(self, sk_bt = None, sk_wk = None, sk_all = None, sk_bl = None):
        
        if sk_bt != None:
            sk_bt = self.team_compos['bt']
        if sk_wk != None:
            sk_wk = self.team_compos['wk']
        if sk_all != None:
            sk_all = self.team_compos['all']
        if sk_bl != None:
            sk_bl = self.team_compos['bl']
        
        # check for sum = 11
        pl_in_team = sk_bt + sk_wk + sk_all + sk_bl
        if pl_in_team != 11:
            print(f"Count of players NOT equal to 11")
        else:
            self.team_compos['bt'] = sk_bt
            self.team_compos['wk'] = sk_wk
            self.team_compos['all'] = sk_all
            self.team_compos['bl'] = sk_bl
            
            # regenerate team with new comp only if update is correct
            self.players = self.generate_team()  
    

### Innings Class

In [74]:
class Innings(Team):
    '''
    Class of Innings object
    Attributes like runs scored, wickets taken, etc. 
    will be part of this class
    '''
    
    def __init__(self, bt_team):
        
        self.bt_team = bt_team
#         self.bl_team = self.second_bt_team if innings==1 else self.first_bt_team
        # batting stats
        self.runs = 0
        self.fours = 0
        self.sixes = 0
        self.boundary = 0
        # bowling stats
        self.wkts = 0
        self.wide = 0
        self.noball = 0
        self.bye = 0
        self.lbye = 0
        self.team_list = self.getPlayerList(bt_team)
    
    
    def getPlayerList(self, bt_team):
        bt_team_o = Team(bt_team)
        return bt_team_o.generate_team()
        
        
    def checkBatter(self):
        '''
        who is batting this ball
        & what is his skill
        '''
        self.batter = self.team_list[self.wkts]
        return self.batter
    
            
    def simulateBall(self):
        '''
        pass
        '''
        pass

### Match Class

In [75]:
class Match(Innings):
    '''
    Class of Match object
    Attributes like teams playing, runs scored, wickets taken, etc. 
    will be part of this class
    '''
    match_cnt = 0
    
    def __init__(self, team1, team2):
        Match.match_cnt += 1
        self.match_id = Match.match_cnt  # id for a match
        self.teams = [team1, team2]
        choice1 = np.random.choice(self.teams, 1)
        choice2 = list(set(self.teams).difference(set(choice1)))
        self.first_bt_team = choice1[0]
        self.second_bt_team = choice2[0]
        self.team_list = {tn: Team(tn) for tn in self.teams}
    
    
    def initiateInnings(self):
        self.inn = {}
        for i in range(1, 3):
            self.inn[i] = Innings(self.first_bt_team) if i == 1 else Innings(self.second_bt_team)
    

# Initialize Match / Innings

In [76]:
all_teams = list(p_names.keys())
all_teams
teams = np.random.choice(all_teams, size = 2, replace = False)
teams
match = Match(team1 = teams[0], team2 = teams[1])

match.match_cnt
match.teams
match.first_bt_team
match.second_bt_team

['ind', 'aus', 'saf', 'win', 'sri']

array(['aus', 'ind'], dtype='<U3')

1

['aus', 'ind']

'aus'

'ind'

In [77]:
inn = match.initiateInnings()

In [78]:
match.inn.keys()
match.inn[1].bt_team
# match.inn[1].

dict_keys([1, 2])

'aus'

In [79]:
match.inn[1].team_list

['jlanger',
 'rponting',
 'mhussey',
 'ssmith',
 'agilchrist',
 'asymonds',
 'mclarke',
 'abitchell',
 'blee',
 'mstark',
 'gmcgrath']

In [120]:
pl = match.inn[1].team_list[2]
pl

'mhussey'

In [121]:
p = Player(pl, match.inn[1].bt_team)

In [122]:
p.runs_proba
p.wkts_proba

{0: 0.5208333333333334,
 1: 0.23674242424242423,
 2: 0.0946969696969697,
 3: 0.004734848484848485,
 4: 0.0946969696969697,
 5: 0.000946969696969697,
 6: 0.04734848484848485}

{0: 0.9991, 1: 0.0009}

In [123]:
# {b: p.simulateScore(wkts=0, balls=b, runs_proba=p.runs_proba, wkts_proba=p.wkts_proba) for b in range(1,20) if p.simulateScore(wkts=0, balls=b, runs_proba=p.runs_proba, wkts_proba=p.wkts_proba) != -1}
for b in range(1,300): 
    run_in_ball = p.simulateScore(wkts=0, balls=b, runs_proba=p.runs_proba, wkts_proba=p.wkts_proba)
    if run_in_ball == -1:
        print('Out! Fall of Wicket')
        break;
#     else:
#         {b: run_in_ball}
#         p.bt_runs
#         f"run in this ball: {run_in_ball}"
#         f"{'-'*5} Runs Probability {'-'*5}"
#         p.runs_proba
{p.bt_balls: p.bt_runs}     

Out! Fall of Wicket


{72: 61}

# Initialize Teams

In [ ]:
team_list = {tn: Team(tn) for tn in match.teams}
team_list.keys()

In [ ]:
list(team_list[match.first_bt_team].players_obj_list.keys())

# Test Simulation

In [ ]:
def simulate_score(wkts, balls, runs_proba, wkts_proba):
    
    balls += 1
    wkts_in_ball = np.random.choice(a = list(wkts_proba.keys()), p = pd.Series(list(wkts_proba.values())))
    if wkts_in_ball:
        wkts += 1
        run_in_ball = -1
        runs_proba = reset_runs_proba()
    else: 
        run_in_ball = np.random.choice(a = list(runs_proba.keys()), p = pd.Series(list(runs_proba.values())))
        
        # runs proba updated as per run scored in prev ball
        for k, v in list(runs_proba.items()):
            if k == run_in_ball:
                if run_in_ball == 0:
                    runs_proba[k] = v*1.05
                else:
                    runs_proba[k] = v*1.2
            elif k > run_in_ball:
                runs_proba[k] = v*1.01
            else:
                runs_proba[k] = v
        
        dict_sum = pd.Series(list(runs_proba.values())).sum()
        runs_proba = {k: v/dict_sum for k, v in list(runs_proba.items())}  # normalize so sum of proba = 1
    
    return (wkts, wkts_in_ball, run_in_ball, balls, runs_proba, wkts_proba)

In [ ]:
def run_bkts(ball_run_dict):
    temp_df = pd.Series(list(ball_run_dict.values())).to_frame()
    temp_df.columns = ['runs']

    score_df = temp_df['runs'].value_counts().to_frame()
    score_df['score'] = score_df.index
    score_df['run_sum'] = score_df['score'] * score_df['runs']

    return score_df

In [ ]:
def innings_simulator():
    wkts = 0
    balls = 0
    runs = 0
    ball_run_dict = {}
    ball_wkt_dict = {}

    wkts_proba = reset_wkts_proba()
    runs_proba = reset_runs_proba()
    
    while wkts <= 10: 
        if balls >= 300:
            break
        else:
            wkts, wkts_in_ball, run_in_ball, balls, runs_proba, wkts_proba = simulate_score(wkts, balls, runs_proba, wkts_proba)
            ball_run_dict[balls] = run_in_ball
            ball_wkt_dict[balls] = wkts_in_ball
    #         f"balls: {balls} | wkts: {wkts} | run: {run_in_ball}"
            runs += run_in_ball if run_in_ball != -1 else 0
    print(f"overall runs: {runs}")
    print(f"balls: {balls}")
    print(f"wkts: {pd.Series(list(ball_wkt_dict.values())).sum()}")
    
    return run_bkts(ball_run_dict), ball_run_dict

In [ ]:
score_df, ball_run_dict = innings_simulator()
score_df

In [ ]:
runs_proba = reset_runs_proba()
runs_proba

In [ ]:
f"{'-'*5} Initial prob {'-'*5}"
{k: round(v, 2) for k, v in list(runs_proba.items())}
run_in_ball = np.random.choice(a = list(runs_proba.keys()), p = pd.Series(list(runs_proba.values())))
run_in_ball
runs_proba = {k: v*1.05 if run_in_ball == 0 else v*1.3 if run_in_ball<=k else v for k, v in runs_proba.items()}
f"{'-'*5} After update prob {'-'*5}"
{k: round(v, 2) for k, v in list(runs_proba.items())}
dict_sum = pd.Series(list(runs_proba.values())).sum()
runs_proba = {k: v/dict_sum for k, v in list(runs_proba.items())}  # normalize so sum of proba = 1
f"{'-'*5} Normalized prob {'-'*5}"
{k: round(v, 2) for k, v in list(runs_proba.items())}